Many quantum algorithms requires the optimization of quantum circuit parameters w.r.t an expectation value. CUDA-Q has a number of tools available for optimization techniques. This example will demonstrate how to opimizae the variational parameters of a circuit using:
1. Built in CUDA-Q optimizers and gradients
2. A Third-Party Opimizer
3. A parallel parameter shift gradient  

First, the kernel and Hamiltonian and specified below

In [44]:
import cudaq
from cudaq import spin
import numpy as np

hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) - 2.1433 * spin.y(0) * spin.y(1) + .21829 * spin.z(0) - 6.125 * spin.z(1) + spin.z(2) - spin.x(2)*spin.x(0)

@cudaq.kernel
def kernel(angles: list[float]):
    qubits = cudaq.qvector(3)
    for q in range(3):
        h(qubits[q])
    
    for q in range(3):
        ry(angles[3*q], qubits[q])
        rz(angles[3*q+1], qubits[q])
        ry(angles[3*q+2], qubits[q])

    #for q in range(3):
     #   x.ctrl(qubits[q], qubits[(q+1)%3])

initial_params = np.random.normal(0, np.pi, 2)

# Built in CUDA-Q Optimizers and Gradients  
An objective function is defined next which uses a lambda expression to evaluate the cost (a CUDA-Q ```observe``` expectation value). The gradient is calculated using the ```compute``` method

In [45]:
optimizer = cudaq.optimizers.Adam()
gradient = cudaq.gradients.CentralDifference()

def objective_function(parameter_vector: list[float],
                       hamiltonian=hamiltonian,
                       gradient_strategy=gradient,
                       kernel=kernel) -> tuple[float, list[float]]:

        get_result = lambda parameter_vector: cudaq.observe(kernel, hamiltonian, parameter_vector).expectation()

        cost = get_result(parameter_vector)

        gradient_vector = gradient_strategy.compute(parameter_vector, get_result, cost)

        return cost, gradient_vector

energy, params = optimizer.optimize(dimensions = 12, function = objective_function)
print(params)
print(f"\nminimized <H> = {round(energy,16)}")
print(f"optimal theta 0 = {round(params[0],16)}")

[0.07631984471016705, 0.0, 0.07631984471016705, -0.6155723944862938, 0.0, -0.6155723944862938, 0.39583465289385406, 0.0, 0.39583465289385406, 0.0, 0.0, 0.0]

minimized <H> = -2.012978798121103
optimal theta 0 = 0.076319844710167


# Third-Party Optimizers  
The same procedure can be accoplishedusing any third-party such as SciPy.

In [22]:
from scipy.optimize import minimize

def cost(theta):
    exp_val = cudaq.observe(kernel, hamiltonian, theta).expectation()

    return exp_val
result = minimize(cost, initial_params, method='COBYLA', options = {'maxiter':40})
result

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -1.7488649713984867
       x: [-5.689e+00 -7.846e-01]
    nfev: 33
   maxcv: 0.0